<a href="https://colab.research.google.com/github/hsaglamlar-test/new-project/blob/main/task_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 pip install -q spark-nlp==3.4.2 pyspark==3.2.0

     |████████████████████████████████| 142 kB 34.0 MB/s 
     |████████████████████████████████| 281.3 MB 33 kB/s 
     |████████████████████████████████| 198 kB 65.8 MB/s 


In [2]:
import sparknlp

spark = sparknlp.start(gpu = True, spark32=True) # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
import os

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 3.4.2
Apache Spark version: 3.2.0


# Cleaning

In [3]:
pd_train = pd.read_csv("df_train.csv")
pd_test = pd.read_csv("df_val.csv")
pd_train


,text,tweet_id,label
0,Depression and anxiety will probably end up ki...,1189514994452520960,1
1,growing accustomed to living in a constant sta...,1199424423758983169,1
2,Ive been feeling like im in a constant state o...,1183952883395547136,1
3,@Screvvy Do you enjoy living constantly in thi...,1206425604607877120,0
4,Me: drained from constant stress\nAlso me: goe...,1184702290801889280,0
...,...,...,...
2931,Having a chronic illness sucks and then gettin...,1197779681547706368,0
2932,why is my life a constant struggle like nothin...,1192044023986479111,1
2933,How the fuck am i supposed to be in any christ...,1199041248407019521,0
2934,"RT ShiftConMedia ""So how can you short circuit...",1180139887959461894,0


In [4]:
%pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 31.7 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=b292b86f1d1ca3c42d59fa90e3817a61ad10d5232a6359942125631617b6c441
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [5]:
import emoji
import re
def give_emoji_free_text(text):
    return emoji.replace_emoji(text, replace='')

def do_process_eng_hashtag(input_text: str) -> str:
    return re.sub(
        r'#[a-z]\S*',
        lambda m: ' '.join(re.findall('[A-Z][^A-Z]*|[a-z][^A-Z]*', m.group().lstrip('#'))),
        input_text,
    )


In [6]:
txt1='ahmet #eveGelHemen sonra gidersin.'
do_process_eng_hashtag(txt1)

'ahmet eve Gel Hemen sonra gidersin.'

In [38]:
def clean_tweet(df_column):

  df_column = df_column.str.replace(r'@[A-Za-z0-9_]+','', regex=True)
  df_column = df_column.str.replace(r'#[A-Za-z0-9_]+','', regex=True)
  df_column = df_column.str.replace(r'&[A-Za-z0-9_]+;','', regex=True)
  df_column = df_column.str.replace(r"http\S+", "", regex=True)
  df_column = df_column.str.replace(r"www.\S+", "", regex=True)
  df_column = df_column.str.replace("\n", " ")

  df_column = df_column.apply(give_emoji_free_text)
  # df_column = df_column.apply(lambda x: do_process_eng_hashtag(x) )

  return df_column

In [24]:
pd_train.text[111]

'loosing 10 lbs in less than a month because you’ve been constantly stressed  lol '

In [42]:
pd_train.text = clean_tweet(pd_train.text)


In [43]:
pd_train.text[4]

'Me: drained from constant stress Also me: goes on social media for hours at a time'

In [27]:
pd_train.text[111]

'loosing 10 lbs in less than a month because you’ve been constantly stressed  lol '

In [14]:
pd_test.text = clean_tweet(pd_test.text)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


In [15]:
df_train= spark.createDataFrame(pd_train)
df_test= spark.createDataFrame(pd_test)
df_test.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+-------------------+-----+
|text                                                                                                                                        |tweet_id           |label|
+--------------------------------------------------------------------------------------------------------------------------------------------+-------------------+-----+
|Diastolic blood pressure measure Chronic Physical Stress.                                                                                   |1202304872848252930|0    |
|i live in constant stress over these cans of soup                                                                                           |1221223724990521344|1    |
|i’m constantly stressed over absolutely NOTHING                                                                                             |1227475850871

# From tokens

In [16]:
document_assembler = DocumentAssembler() \
      .setInputCol("text") \
      .setOutputCol("document")
    
tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")\
      .setLowercase(True)\


stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

# lemma = LemmatizerModel.pretrained('lemma_antbnc') \
#       .setInputCols(["cleanTokens"]) \
#       .setOutputCol("lemma")

glove_embeddings = WordEmbeddingsModel().pretrained() \
      .setInputCols(["document",'cleanTokens'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("prediction")\
      .setLabelColumn("label")\
      .setMaxEpochs(10)\
      .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            # lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierdl])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [17]:
clf_pipelineModel = clf_pipeline.fit(df_train)

In [18]:
pd_result = clf_pipelineModel.transform(df_test).select('text','label',"prediction.result").toPandas()
pd_result['result'] = pd_result['result'].apply(lambda x: x[0]).astype('int')
pd_result.head(20)

,text,label,result
0,Diastolic blood pressure measure Chronic Physi...,0,0
1,i live in constant stress over these cans of s...,1,0
2,i’m constantly stressed over absolutely NOTHING,1,0
3,I genuinely cannot wait to be done with colleg...,1,0
4,I don’t wanna die from chronic stress,0,0
5,"With all this constant stress, I barely have t...",1,0
6,i have been in a constant state of stress sinc...,1,0
7,the fact that im constantly getting older stre...,0,0
8,I cannot sleep due to my constant stress. I ca...,1,0
9,Naaaa this winter season is all cute until ur ...,0,0


In [19]:
pd_result.result.value_counts()

0    420
Name: result, dtype: int64



```
with stop words
              precision    recall  f1-score   support

           0       0.77      0.83      0.80       264
           1       0.67      0.59      0.63       156

    accuracy                           0.74       420
   macro avg       0.72      0.71      0.71       420
weighted avg       0.74      0.74      0.74       420
```



In [20]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(pd_result.label, pd_result.result))
print(accuracy_score(pd_result.label, pd_result.result))

              precision    recall  f1-score   support

           0       0.63      1.00      0.77       264
           1       0.00      0.00      0.00       156

    accuracy                           0.63       420
   macro avg       0.31      0.50      0.39       420
weighted avg       0.40      0.63      0.49       420

0.6285714285714286


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# USE embeddings

In [21]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained()\
      .setInputCols("document")\
      .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")\
    .setLabelColumn("label")\
    .setMaxEpochs(5)\
    .setEnableOutputLogs(True)

use_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [22]:
use_pipelineModel = use_pipeline.fit(df_train)

In [23]:
pd_result_train = use_pipelineModel.transform(df_train).select('text','label',"prediction.result").toPandas()
pd_result_train['result'] = pd_result_train['result'].apply(lambda x: x[0]).astype('int')
pd_result_train.head(20)

,text,label,result
0,Depression and anxiety will probably end up ki...,1,1
1,growing accustomed to living in a constant sta...,1,1
2,Ive been feeling like im in a constant state o...,1,1
3,Do you enjoy living constantly in this self-i...,0,0
4,Me: drained from constant stress\nAlso me: goe...,0,0
5,I’m literally in a constant state of stress an...,1,1
6,hopefully holiday break will make my bf feel a...,0,0
7,"Dissecting connections between chronic stress,...",0,0
8,perrie constantly checking if she had a nip sl...,0,0
9,my body has been emanating constant and excess...,0,0


In [24]:
pd_result = use_pipelineModel.transform(df_test).select('text','label',"prediction.result").toPandas()
pd_result['result'] = pd_result['result'].apply(lambda x: x[0]).astype('int')
pd_result.head(20)

,text,label,result
0,Diastolic blood pressure measure Chronic Physi...,0,0
1,i live in constant stress over these cans of s...,1,0
2,i’m constantly stressed over absolutely NOTHING,1,1
3,I genuinely cannot wait to be done with colleg...,1,1
4,I don’t wanna die from chronic stress,0,1
5,"With all this constant stress, I barely have t...",1,1
6,i have been in a constant state of stress sinc...,1,1
7,the fact that im constantly getting older stre...,0,0
8,I cannot sleep due to my constant stress. I ca...,1,1
9,Naaaa this winter season is all cute until ur ...,0,0


In [25]:
#test scores
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(pd_result.label, pd_result.result))
print(accuracy_score(pd_result.label, pd_result.result))

              precision    recall  f1-score   support

           0       0.85      0.74      0.79       264
           1       0.64      0.78      0.70       156

    accuracy                           0.75       420
   macro avg       0.74      0.76      0.74       420
weighted avg       0.77      0.75      0.76       420

0.7523809523809524


In [26]:
#train scores
print(classification_report(pd_result_train.label, pd_result_train.result))
print(accuracy_score(pd_result_train.label, pd_result_train.result))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85      1844
           1       0.74      0.79      0.76      1092

    accuracy                           0.82      2936
   macro avg       0.80      0.81      0.81      2936
weighted avg       0.82      0.82      0.82      2936

0.8170980926430518


# Bert embeddings

In [27]:
document_assembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")
        
bert_sent = BertSentenceEmbeddings.pretrained("sent_biobert_clinical_base_cased")\
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")

classsifier_dl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("prediction")\
      .setLabelColumn("label")\
      .setMaxEpochs(128)\
      .setBatchSize(8)\
      .setEnableOutputLogs(True)\
      .setLr(0.0005)\
      .setDropout(0.2)

bert_sent_clf_pipeline = Pipeline(
    stages = [
        document_assembler,
        bert_sent,
        classsifier_dl
    ])

sent_biobert_clinical_base_cased download started this may take some time.
Approximate size to download 386.6 MB
[OK!]


In [28]:
bert_pipelineModel = bert_sent_clf_pipeline.fit(df_train)

In [29]:
pd_result = bert_pipelineModel.transform(df_test).select('text','label',"prediction.result").toPandas()
pd_result['result'] = pd_result['result'].apply(lambda x: x[0]).astype('int')
pd_result.head(20)

,text,label,result
0,Diastolic blood pressure measure Chronic Physi...,0,0
1,i live in constant stress over these cans of s...,1,0
2,i’m constantly stressed over absolutely NOTHING,1,1
3,I genuinely cannot wait to be done with colleg...,1,1
4,I don’t wanna die from chronic stress,0,1
5,"With all this constant stress, I barely have t...",1,1
6,i have been in a constant state of stress sinc...,1,1
7,the fact that im constantly getting older stre...,0,0
8,I cannot sleep due to my constant stress. I ca...,1,1
9,Naaaa this winter season is all cute until ur ...,0,0


In [30]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(pd_result.label, pd_result.result))
print(accuracy_score(pd_result.label, pd_result.result))

              precision    recall  f1-score   support

           0       0.79      0.85      0.82       264
           1       0.71      0.62      0.66       156

    accuracy                           0.76       420
   macro avg       0.75      0.73      0.74       420
weighted avg       0.76      0.76      0.76       420

0.7619047619047619


# Longformer

In [32]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

lf_embeddings = LongformerEmbeddings.pretrained("clinical_longformer", "en")\
  .setInputCols(["document", "token"])\
  .setOutputCol("embeddings")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(512)

embeddingsSentence = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")\
    .setLabelColumn("label")\
    .setBatchSize(8)\
    .setEnableOutputLogs(True)\
    .setMaxEpochs(64)\
    .setLr(0.003)\
    .setDropout(0.3)

lf_pipeline = Pipeline(
               stages = [
                    document_assembler,
                    tokenizer,
                    lf_embeddings,
                    embeddingsSentence,
                    classifierdl
               ])

ConnectionRefusedError: ignored

In [ ]:
lf_pipelineModel = lf_pipeline.fit(df_train)

In [ ]:
pd_result = lf_pipelineModel.transform(df_test).select('text','label',"prediction.result").toPandas()
pd_result['result'] = pd_result['result'].apply(lambda x: x[0]).astype('int')
pd_result.head(20)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(pd_result.label, pd_result.result))
print(accuracy_score(pd_result.label, pd_result.result))